# `CLOUD.jl` - 2D linear advection example

In [43]:
using CLOUD, OrdinaryDiffEq

Define the physical problem

In [44]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [45]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm()
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [46]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [47]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         86.2μs / 718147.1%        10.6KiB / 6255254.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          40.8k    387ms   62.6%  9.48μs    293MiB   45.4%  7.34KiB
   volume terms         40.8k    162ms   26.3%  3.98μs    156MiB   24.1%  3.91KiB
   facet terms          40.8k    124ms   20.1%  3.05μs   86.6MiB   13.4%  2.17KiB
   mass matrix solve    40.8k   45.4ms    7.3%  1.11μs   19.9MiB    3.1%     512B
 extrap solution        40.8k   95.6ms   15.4%  2.34μs   71.0MiB   11.0%  1.78KiB
 eval num flux          40.8k   51.9ms    8.4%  1.27μs    106MiB   16.4%  2.66KiB
 gather ext sta

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [48]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.005749119046586659
Conservation (initial/final/diff):
[8.02358962898736e-8][8.023589453433344e-8][-1.7555401576885288e-15]
Energy (initial/final/diff):
[0.124998741370523][0.12439699898414937][-0.0006017423863736299]


Write output to VTK file

In [49]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));